In [1]:
import gurobipy as gp
import numpy as np
import time

from DaTaReachControl.idealisticControlGrb import *
from DaTaReachControl.idealisticControlAPGDAR import *
from DaTaReachControl.optimisticControlGrb import *

In [9]:
# val = int(np.random.uniform(0,4000))
# print (val)
# np.random.seed(val)
# np.random.seed(2386)
nS = 5
nC = 3

A1_lb = np.random.random((nS,nC))
A1_ub = A1_lb + 0.1*np.random.random((nS,nC))
A2_lb = A1_lb
A2_ub = A1_ub
b_lb = np.random.random(nS)
b_ub = b_lb + 0.01 * np.random.random(nS)

Q = np.random.random((nS,nS))
Q = Q @ Q.T
q = np.random.random(nS)
R = np.zeros((nC,nC), dtype=np.float64)
S = np.zeros((nS, nC), dtype=np.float64)
r = np.zeros(nC, dtype=np.float64)

u_lb = (np.random.random(nC) - 0.5) * 2
u_ub = u_lb + np.random.random(nC)

w1 = np.random.random()
w2 = np.random.random()
w3 = np.random.random()

In [30]:
u_lb[2] = -u_lb[2]
print (u_lb, u_ub)

[ 0.05035154 -0.01709445 -0.11797619] [0.36259868 0.31701789 1.00362112]


In [11]:
# Midpoint with GUrbobi
initIdealisticProblemGrb(u_lb, u_ub)

In [38]:
tC = time.time()
uGrb, cGrb = solveIdealisticProblemGrb(A1_lb, A1_ub, A2_lb, A2_ub, b_lb, b_ub, u_lb, u_ub,
                            1, Q, S, R, q, r, w1, w2, w3, verbose=False)
print(time.time() - tC)
print(uGrb)
print(cGrb)

0.0018947124481201172
[ 0.05035154 -0.01709445  0.        ]
0.4667641848391332


In [39]:
# Midpoint with Accelerated Projected Gradient With Gradient Restart Scheme
tC = time.time()
uAPG, cAPG = solveIdealisticProblemAPGDAR(A1_lb, A1_ub, A2_lb, A2_ub, b_lb, b_ub, u_lb, u_ub,
                            1, Q, S, R, q, r, w1, w2, w3, epsTol=1e-8)
print(time.time() - tC)
print(uAPG)
print(cAPG)

2
0.0023581981658935547
[ 0.05035154 -0.01709445  0.        ]
0.46676418394140695


In [14]:
# Check the optimistic problem
initOptimisticProblemGrb(nS, nC, None, None, None, None, None, u_lb, u_ub)

In [40]:
updateCost(Q, S, R, q, r)
tC = time.time()
uOp, cOp = solveOptimisticProblemGrb(A1_lb, A1_ub, A2_lb, A2_ub, b_lb, b_ub,
            u_lb, u_ub, 1, verbose=False)
print(time.time() - tC)
print(uOp, cOp)

0.0024695396423339844
[ 0.05035154 -0.01709445  0.        ] 5.299882334688636
